# asmFoam Verification Case

This case compares solutions of the ASM1 biokinetics equations, solved as an ODE and using the `asmFOAM` CFD solver. The CFD case includes both flow and diffusion, but the oxygen transfer model is based on a constant `kLa` parameter. Provided the vessel is well mixed, the solution should match the ODE solution.

To run the case, first solve the flow field by running the case in the `Flow` directory. The process to run the case is to run `blockMesh`, followed by `setFields`, followed by `reactingTwoPhaseEulerFoam`. Then, copy the files `alpha.airMean`, `U.airMean`, and `U.waterMean` into the `0` directory of the case in the `Biokinetics` directory. Be sure the rename them to `alpha.air`, `U.air`, and `U.water`, resepectively. In this case, the flow is not updated from the initial condition. Although there are many other variables that are solved as a part of the flow field, these three are the only ones upon which the biokinetics equations depend on directly. To run the biokinetics solver, simply run `blockMesh`, followed by `asmFOAM`. Running this notebook will read the volume-averaged solution fields and compare them with the solution of the corresponding ODE system.

In [ ]:
# Define the ASM model constants

# Stoichiometry (all dimensionless)

YA  = 0.24;      # Heterotrophic yield
YH  = 0.67;      # Autotrophic yield
fP  = 0.08;      # Fraction of biomass yielding particulate products
iXB = 0.086;     # Mass of nitrogen per mass of COD in biomass
iXP = 0.06;      # Mass of nitrogen per mass of COD in products from biomass

# Kinetics parameters
muH   = 3.0;     # Max. specific growth rate for heterotrophic biomass [1/day]
KS    = 20;      # Half-saturation coefficient for heterotrophic biomass [g/m^3]
KOH   = 0.2;     # Oxygen half-saturation coefficient for heterotrophic biomass [g/m^3]
KNO   = 0.5;     # Nitrate half-saturation coefficient for denitrifying heterotrophic biomass [g/m^3]
bH    = 0.42;    # Decay coefficient for heterotrophic biomass [1/day]
etag  = 0.8;     # Correction factor for muH under anoxic conditions []
etah  = 0.4;     # Correction factor for hydrolysis under anoxic conditions []
kh    = 2.0;     # Maximum specific hydrolysis rate [1/day]
KX    = 0.02;    # Half-saturation coefficient for hydrolysis of slowly biodegradable substrate []
muA   = 0.5;     # Maximum specific growth rate for autotrophic biomass [1/day]
KNH   = 1;       # Ammonia half-saturation coefficient for autotrophic biomass [g/m^3]
KOA   = 0.4;     # Oxygen half-saturation coefficient for autotrophic biomass [g/m^3]
bA    = 0.42;    # Decay coefficient for autotrophic biomass [1/day]
ka    = 0.08;    # Ammonification rate [m^3/g/day]
SOsat = 9.28;    # Oxygen saturation concentration [g/m^3]


In [ ]:
# Read the CFD data

import csv
import pandas as pd

# Read the data file
cfd_data = pd.read_csv('./postProcessing/asmFieldAverages/0/volFieldValue_30.dat', sep='\t', skiprows=4)
cfd_data.columns = ["time", "SS", "XS", "XBH", "XBA", "XP", "SO", "SNO", "SNH", "SND", "XND", "kLa"]

# Convert units to days and grams/m^3
cfd_data["time"] /= 86400
cfd_data[["SS", "XS", "XBH", "XBA", "XP", "SO", "SNO", "SNH", "SND", "XND"]] *= 1000
cfd_data["kLa"] *= 86400

In [ ]:
# Solve the ODE system

import numpy as np
import pandas as pd
from scipy.integrate import ode
from scipy.interpolate import interp1d

# Define the initial concentrations and state vector

SS0 = 580
XS0 = 360
XBH0 = 500
XBA0 = 50
XP0 = 40
SO0 = 0.5
SNO0 = 1
SNH0 = 60
SND0 = 6
XND0 = 15

# Set up the interpolation object to calculate kLa values
kLa_interp = interp1d(cfd_data["time"], cfd_data["kLa"],
                      bounds_error=False,
                      fill_value=cfd_data["kLa"][0])

x0 = np.array([SS0, XS0, XBH0, XBA0, XP0, SO0, SNO0, SNH0, SND0, XND0])

# Define ODE source terms
def f(t, x):
    # Get the state variables
    SS = x[0]
    XS = x[1]
    XBH = x[2]
    XBA = x[3]
    XP = x[4]
    SO = x[5]
    SNO = x[6]
    SNH = x[7]
    SND = x[8]
    XND = x[9]
    
    # Interpolate the kLa value from the CFD data
    kLa = kLa_interp(t)
    
    # Define the reaction rates
    rhoHeterotrophAerobicGrowth = muH * SS/(KS + SS) * SO/(KOH + SO) * XBH
    rhoHeterotrophAnoxicGrowth = muH * SS/(KS + SS) * KOH/(KOH + SO) * SNO/(KNO + SNO) \
                               * etag * XBH
    rhoAutotrophAerobicGrowth = muA * SNH/(KNH + SNH) * SO/(KOA + SO) * XBA
    rhoHeterotrophDecay = bH * XBH
    rhoAutotrophDecay = bA * XBA
    rhoAmmonification = ka * SND * XBH
    rhoOrganicHydrolysis = kh * XS/XBH/(KX + XS/XBH) * (SO/(KOH + SO) \
                         + etah * KOH/(KOH + SO) * SNO/(KNO + SNO)) * XBH
    rhoOrganicNitrogenHydrolysis = rhoOrganicHydrolysis * XND / XS
    
    # Define the source term
    src = np.zeros(10)
    src[0] = - rhoHeterotrophAerobicGrowth/YH \
             - rhoHeterotrophAnoxicGrowth/YH \
             + rhoOrganicHydrolysis
    src[1] = + (1-fP)*rhoHeterotrophDecay \
             + (1-fP)*rhoAutotrophDecay \
             - rhoOrganicHydrolysis
    src[2] = + rhoHeterotrophAerobicGrowth \
             + rhoHeterotrophAnoxicGrowth \
             - rhoHeterotrophDecay
    src[3] = + rhoAutotrophAerobicGrowth \
             - rhoAutotrophDecay
    src[4] = + fP*rhoHeterotrophDecay \
             + fP*rhoAutotrophDecay
    src[5] = - (1-YH)/YH*rhoHeterotrophAerobicGrowth \
             - (4.57-YA)/YA*rhoAutotrophAerobicGrowth \
             + kLa*(SOsat - SO)
    src[6] = - (1-YH)/2.86/YH*rhoHeterotrophAnoxicGrowth \
             + rhoAutotrophAerobicGrowth/YA
    src[7] = - iXB*rhoHeterotrophAerobicGrowth \
             - iXB*rhoHeterotrophAnoxicGrowth \
             - (iXB+1/YA)*rhoAutotrophAerobicGrowth \
             + rhoAmmonification
    src[8] = - rhoAmmonification \
             + rhoOrganicNitrogenHydrolysis
    src[9] = + (iXB-fP*iXP)*rhoHeterotrophDecay \
             + (iXB-fP*iXP)*rhoAutotrophDecay \
             - rhoOrganicNitrogenHydrolysis
    
    # Return the source term
    return src

# Set time information
dt = 0.001   # [days]
tStart = 0.0 # [days]
tEnd = 0.5   # [days]

# Set up the ODE integrator
r = ode(f)
r.set_initial_value(x0, tStart)

# Set up data frame for results
ode_data = pd.DataFrame(columns=["time", 
                                 "SS", "XS", "XBH", "XBA", "XP", 
                                 "SO", "SNO", "SNH", "SND", "XND"],
                        index=range(0, int((tEnd-tStart)/dt)),
                        dtype='float')

# Solve ODE
i = 0
while r.successful() and r.t < tEnd:
    r.integrate(r.t+dt)
    ode_data.loc[i] = [r.t] + [r.y[j] for j in range(len(r.y))]
    i += 1

In [ ]:
# Plot the oxygen transfer coefficient (kLa)

%matplotlib inline
import matplotlib.pyplot as plt
plt.plot(cfd_data["time"], cfd_data["kLa"], 'c-', label='CFD')
plt.legend()
plt.xlabel(r'$t$ [days]')
plt.ylabel(r'$k_La$ [days$^{-1}$]')
plt.show()

In [ ]:
# Plot the readily biodegradable substrate (SS)

%matplotlib inline
import matplotlib.pyplot as plt
plt.plot(ode_data["time"], ode_data["SS"], 'b--', label='ODE')
plt.plot(cfd_data["time"], cfd_data["SS"], 'c-', label='CFD')
plt.legend()
plt.xlabel(r'$t$ [days]')
plt.ylabel(r'$S_S$ [g/m$^3$]')
plt.show()

In [ ]:
# Plot the slowly biodegradable substrate (XS)

%matplotlib inline
import matplotlib.pyplot as plt
plt.plot(ode_data["time"], ode_data["XS"], 'b--', label='ODE')
plt.plot(cfd_data["time"], cfd_data["XS"], 'c-', label='CFD')
plt.legend()
plt.xlabel(r'$t$ [days]')
plt.ylabel(r'$X_S$ [g/m$^3$]')
plt.show()

In [ ]:
# Plot the active heterotrophic biomass (XBH)

%matplotlib inline
import matplotlib.pyplot as plt
plt.plot(ode_data["time"], ode_data["XBH"], 'b--', label='ODE')
plt.plot(cfd_data["time"], cfd_data["XBH"], 'c-', label='CFD')
plt.legend()
plt.xlabel(r'$t$ [days]')
plt.ylabel(r'$X_{B,H}$ [g/m$^3$]')
plt.show()

In [ ]:
# Plot the active autotrophic biomass (XBA)

%matplotlib inline
import matplotlib.pyplot as plt
plt.plot(ode_data["time"], ode_data["XBA"], 'b--', label='ODE')
plt.plot(cfd_data["time"], cfd_data["XBA"], 'c-', label='CFD')
plt.legend()
plt.xlabel(r'$t$ [days]')
plt.ylabel(r'$X_{B,A}$ [g/m$^3$]')
plt.show()

In [ ]:
# Plot the particulate products arising from biomass decay (XP)

%matplotlib inline
import matplotlib.pyplot as plt
plt.plot(ode_data["time"], ode_data["XP"], 'b--', label='ODE')
plt.plot(cfd_data["time"], cfd_data["XP"], 'c-', label='CFD')
plt.legend()
plt.xlabel(r'$t$ [days]')
plt.ylabel(r'$X_P$ [g/m$^3$]')
plt.show()

In [ ]:
# Plot the oxygen (SO)

%matplotlib inline
import matplotlib.pyplot as plt
plt.plot(ode_data["time"], ode_data["SO"], 'b--', label='ODE')
plt.plot(cfd_data["time"], cfd_data["SO"], 'c-', label='CFD')
plt.legend()
plt.xlabel(r'$t$ [days]')
plt.ylabel(r'$S_O$ [g/m$^3$]')
plt.show()

In [ ]:
# Plot the nitrate and nitrite (SNO)

%matplotlib inline
import matplotlib.pyplot as plt
plt.plot(ode_data["time"], ode_data["SNO"], 'b--', label='ODE')
plt.plot(cfd_data["time"], cfd_data["SNO"], 'c-', label='CFD')
plt.legend()
plt.xlabel(r'$t$ [days]')
plt.ylabel(r'$S_{NO}$ [g/m$^3$]')
plt.show()

In [ ]:
# Plot the ammonia and ammonium nitrogen (SNH)

%matplotlib inline
import matplotlib.pyplot as plt
plt.plot(ode_data["time"], ode_data["SNH"], 'b--', label='ODE')
plt.plot(cfd_data["time"], cfd_data["SNH"], 'c-', label='CFD')
plt.legend()
plt.xlabel(r'$t$ [days]')
plt.ylabel(r'$S_{NH}$ [g/m$^3$]')
plt.show()

In [ ]:
# Plot the soluble biodegradable organic nitrogen (SND)

%matplotlib inline
import matplotlib.pyplot as plt
plt.plot(ode_data["time"], ode_data["SND"], 'b--', label='ODE')
plt.plot(cfd_data["time"], cfd_data["SND"], 'c-', label='CFD')
plt.legend()
plt.xlabel(r'$t$ [days]')
plt.ylabel(r'$S_{ND}$ [g/m$^3$]')
plt.show()

In [ ]:
# Plot the particulate biodegradable organic nitrogen (XND)

%matplotlib inline
import matplotlib.pyplot as plt
plt.plot(ode_data["time"], ode_data["XND"], 'b--', label='ODE')
plt.plot(cfd_data["time"], cfd_data["XND"], 'c-', label='CFD')
plt.legend()
plt.xlabel(r'$t$ [days]')
plt.ylabel(r'$X_{ND}$ [g/m$^3$]')
plt.show()